**利用gdal开源库进行栅格文件处理**


In [13]:
from osgeo import gdal, osr


1. 重投影

In [14]:
path_img = 'data/Section-2/s2_kunming_chenggong_4bands_10m.tif'
path_reproj = 'data/Section-5/s2_kunming_chenggong_4bands_10m_wgs84.tif'


In [15]:
## 读入栅格数据，包括影像信息和地理信息
dset = gdal.Open(path_img)
proj = dset.GetProjection()    ### 获取影像投影
print('projection:', proj)     ### wkt格式投影表达


projection: PROJCS["WGS 84 / UTM zone 48N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",105],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32648"]]


In [16]:
### 左上角utm坐标计算, 及utm投影下影像范围
## 定义空间参考
utm_epsg = 32648
wgs84_epsg = 4326
utm = osr.SpatialReference(); 
utm.ImportFromEPSG(utm_epsg)        ## 定义空间参考
wgs84 = osr.SpatialReference(); 
wgs84.ImportFromEPSG(wgs84_epsg)    ## 定义空间参考


0

In [17]:
### 投影转换后像元尺寸（分辨率）计算
## 获得空间参考转换后角点坐标
wgs84.SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER)  ### 注意！设置坐标转换输出次序:（x, y）
transform = osr.CoordinateTransformation(utm, wgs84)    ### 空间参考转换对象
geo_trans = dset.GetGeoTransform()   ### 地理转换参数
print(geo_trans[0], geo_trans[3])    ### 左上角x,y坐标
(x_min_wgs84, y_max_wgs84, z) = transform.TransformPoint(geo_trans[0], geo_trans[3], 0)  ## 输入参数为(经度，纬度，高程）, 输出参数为（x，y），注意顺序
print('the left-up coordinate of the wgs84 projection:', x_min_wgs84, y_max_wgs84)
x_max_utm = geo_trans[0]+geo_trans[1]*dset.RasterXSize   ## x方向最大值
y_min_utm = geo_trans[3]+geo_trans[5]*dset.RasterYSize   ## y方向最小值
(x_max_wgs84, y_min_wgs84, z) = transform.TransformPoint(x_max_utm, y_min_utm, 0)  ## 输入参数为(x，y), 输出参数为（x，y），注意顺序
print(x_max_wgs84, y_min_wgs84)
## 计算空间参考转换后像元尺寸
width_pixel = (x_max_wgs84-x_min_wgs84)/dset.RasterXSize
height_pixel = (y_max_wgs84-y_min_wgs84)/dset.RasterYSize
print(width_pixel, height_pixel)


268170.0 2765450.0
the left-up coordinate of the wgs84 projection: 102.70324341425528 24.98680372054124
103.00458005060858 24.700098045073705
0.00010057965165330377 8.89285593881929e-05


In [ ]:
### 定义新的转换参数和投影
#### 1. 新的转换参数, 注意像元高度为负数
geotrans_wgs84 = (102.70324341425528, 0.00010057965165330377, 0.0, 24.98680372054124, 0, -8.89285593881929e-05)
#### 2. 新的投影
proj_utm_wkt = utm.ExportToWkt()
proj_wgs84_wkt = wgs84.ExportToWkt()


In [19]:
### 定义数据驱动
driver = gdal.GetDriverByName('GTiff')
### 创建写出数据
dset_wgs84 = driver.Create(path_reproj, xsize=dset.RasterXSize, \
                                    ysize=dset.RasterYSize, \
                                    bands=dset.RasterCount, \
                                    eType=gdal.GDT_Int16)
dset_wgs84.SetGeoTransform(geotrans_wgs84)
dset_wgs84.SetProjection(proj_wgs84_wkt)

## 影像重投影
reproj = gdal.ReprojectImage(src_ds=dset, \
                            dst_ds=dset_wgs84, \
                            src_wkt=proj_utm_wkt, \
                            dst_wkt=proj_wgs84_wkt,
                            eResampleAlg=gdal.GRA_Bilinear)
dset_wgs84 = None  ## 关闭文件



2. 重采样

In [20]:
path_resam = 'data/Section-5/s2_kunming_chenggong_4bands_10m_resam25m.tif'
res=25  ## 定义分辨率


In [21]:
dset = gdal.Open(path_img)
geo_trans = dset.GetGeoTransform()
x_min, y_max = geo_trans[0], geo_trans[3]
x_max = x_min + geo_trans[1]*dset.RasterXSize
y_min = y_max + geo_trans[5]*dset.RasterYSize
print((x_min, y_max), (x_max, y_min))


(268170.0, 2765450.0) (298130.0, 2733210.0)


In [22]:
### 1.计算影像尺寸
x_size = (x_max - x_min)/res  ## x方向尺寸
y_size = (y_max - y_min)/res   ## y方向尺寸
print(x_size, y_size)
x_size_update, y_size_update = round(x_size), round(y_size)
print(x_size_update, y_size_update)


1198.4 1289.6
1198 1290


In [23]:
### 3. 定义地理转换参数
geotrans_resample = (268170.0, res, 0, 2765450.0, 0, -res)


In [24]:
driver = gdal.GetDriverByName('GTiff')
dset_resam = driver.Create(path_resam, xsize=x_size_update, \
                                ysize=y_size_update, \
                                bands=dset.RasterCount, \
                                eType=gdal.GDT_Int16)
dset_resam.SetGeoTransform(geotrans_resample)
dset_resam.SetProjection(dset.GetProjection())
## 影像重投影
reproj = gdal.ReprojectImage(src_ds=dset, \
                             dst_ds=dset_resam, \
                            eResampleAlg=gdal.GRA_Bilinear)
dset_resam = None  ## 关闭文件


思考：检查重采样影像空间范围，是否发生变化，为什么？